# Topic Modeling

## Introduction

The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and 
see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [ ]:
# Let's read in our document-term matrix
data2 = pd.read_pickle('data_post_stop.pkl')
data2

In [ ]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

In [ ]:
# One of the required inputs is a term-document matrix
termdm = data2.transpose()
termdm = termdm.groupby(level=0, axis=1).sum()
termdm.head()

In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(termdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
import pickle
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda_topics_passes = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda_topics_passes.print_topics()

In [ ]:
# LDA for num_topics = 3
lda_topics_passes = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda_topics_passes.print_topics()

In [ ]:
# LDA for num_topics = 4
lda_topics_passes = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda_topics_passes.print_topics()

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

# Topic Modeling - Attempt #2 (Nouns Only)

In [ ]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    word_noun = lambda position: position[:2] == 'NN'
    tokenized = word_tokenize(text)
    noun_words = [word for (word, position) in pos_tag(tokenized) if word_noun(position)] 
    return ' '.join(noun_words)

In [ ]:
# Read in the cleaned data, before the CountVectorizer step
data_cleaning = pd.read_pickle('data_cleaning.pkl')
data_cleaning

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_cleaning.posts.apply(nouns))
data_nouns

In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['im','like', 'think', 'people', 'dont', 'know', 'really', 'would', 'one', 'get',
 'feel', 'love', 'time', 'ive', 'much', 'say', 'something', 'good', 'start', 'girl',
 'things', 'want', 'see', 'way', 'someone', 'also', 'well', 'friends',
 'always', 'type', 'lot', 'could', 'make', 'go', 'thing', 'even', 'person', 'need',
 'find', 'right', 'never', 'youre', 'thats', 'going', 'life', 'friend',
 'pretty', 'though', 'sure', 'said', 'cant', 'first', 'actually', 'still', 'best', 'many', 'take', 'others', 'work', 'read', 'sometimes', 'got',
 'around', 'thought', 'try', 'back', 'makes', 'better', 'trying', 'didnt', 'agree', 'kind', 'mean', 'tell', 'post', 'two', 'probably', 'talk',
 'anything', 'since', 'maybe', 'understand', 'seems', 'ill', 'id', 'little', 'doesnt', 'thread', 'new', 'long', 'ever', 'years', 'hard', 'might',
 'types', 'us', 'everyone','different', 'look', 'usually', 'may', 'day', 'give', 'come', 'personality', 'guess', 'mind', 'relationship', 'bit', 'quite',
 'great', 'made', 'thinking', 'everything', 'school', 'seem', 'bad', 'every',
 'help', 'yes', 'definitely', 'believe', 'point', 'used', 'infp', 'guys', 'tend','hes', 'use', 'intj', 
 'often', 'getting', 'interesting', 'last', 'talking', 'infj', 'times', 'another', 'mbti', 'enfp', 'world','question','part', 'theres',
 'feeling', 'fun', 'intp', 'enough', 'isnt', 'else', 'hate', 'lol', 'keep',
 'anyone', 'nice', 'idea', 'sense','least','enfj', 'entj', 'entp', 'esfj', 'esfp', 'estj', 'estp',
 'isfj', 'isfp', 'istj', 'istp','sound','thank','enfjs', 'entjs', 'entps', 'esfjs', 'esfps', 'estjs', 'estps',
 'isfjs', 'isfps', 'istjs', 'istps','infjs','infps','intjs','enfps','intps','isnts', 'let', 'problem','im', 'fe',
 'feelings', 'happy', 'thanks', 'sorry', 'care', 'especially', 'guy', 'true', 'experience', 'almost',
 'hope', 'wrong', 'sounds', 'ask', 'close', 'stuff', 'however', 'far', 'strong', 'let', 'theyre', 'put', 'shes',
 'high', 'told', 'year', 'wanted', 'music', 'comes', 'away', 'nothing', 'found', 'head', 'forum', 'start', 'totally',
 'making', 'either', 'test', 'without', 'relate', 'enjoy', 'using', 'rather', 'wouldnt', 'exactly', 'reading', 'big',
 'reason', 'fact', 'real', 'completely', 'problem', 'girl', 'remember', 'social', 'wish', 'change', 'welcome', 'looking',
 'interested', 'ni', 'past', 'along', 'whole', 'able', 'situation', 'emotions', 'old', 'figure', 'similar', 'done', 'show',
 'advice', 'havent', 'functions', 'important', 'felt', 'already', 'met', 'alone', 'saying', 'yet', 'seen', 'emotional', 'end',
 'family', 'place', 'book', 'ago', 'hear', 'open', 'example', 'live', 'sort', 'man', 'fi', 'oh', 'together', 'awesome', 'relationships',
 'ones', 'words', 'personally', 'arent', 'stop', 'mostly', 'matter', 'certain', 'job', 'crazy', 'wont', 'wasnt', 'answer', 'general',
 'took', 'although', 'honestly', 'gets', 'meet', 'months', 'means', 'group', 'working', 'started', 'recently', 'needs', 'course',
 'mom', 'days', 'conversation', 'less', 'based', 'questions', 'dad', 'easy', 'taking', 'interest', 'funny', 'saw', 'weird', 'depends',
 'posts', 'cool', 'mine', 'case', 'enneagram', 'extremely', 'goes', 'honest', 'hurt', 'youve', 'ne', 'night', 'next', 'consider', 'opinion',
 'whatever', 'absolutely', 'home', 'learn', 'explain', 'topic', 'prefer', 'play', 'appreciate', 'name', 'heard', 'must', 'seriously', 'self', 
 'likely', 'function', 'tried', 'couple', 'easily', 'wants', 'generally', 'week', 'known', 'ok', 'asking', 'trust', 'list', 'eyes', 'taken',
 'common', 'second', 'unless', 'posted', 'speak', 'write', 'instead', 'stay', 'meant', 'attention', 'ways', 'free', 'whether', 'college', 
 'female', 'seeing', 'says', 'parents', 'writing', 'fine', 'half', 'si', 'stand', 'god', 'watch', 'haha', 'today', 'personal', 'yeah', 'share',
 'become', 'thoughts', 'feels', 'went', 'favorite', 'side', 'kinda', 'curious', 'whats', 'deep', 'towards', 'please', 'video', 'asked',
 'possible', 'takes', 'act', 'problems', 'deal', 'etc', 'se', 'male', 'call', 'knew', 'wonder', 'women', 'dating', 'happens', 'face', 'intjs',
 'lets', 'works', 'story', 'perfect', 'anyway', 'realize', 'move', 'coming', 'introverted', 'word', 'mother', 'okay', 'movie', 'currently',
 'add', 'leave', 'happen', 'ideas', 'listen', 'noticed', 'came', 'game', 'called', 'picture', 'looks', 'stupid', 'understanding', 'issue',
 'level', 'issues', 'girls', 'liked', 'future', 'house', 'watching', 'money', 'spend', 'notice', 'three', 'class', 'short', 'view', 'cognitive',
 'te', 'order', 'sent', 'moment', 'difficult', 'left', 'sex', 'information', 'lots', 'cause', 'basically', 'fit', 'couldnt', 'ii',
 'happened', 'except', 'response', 'reasons', 'full', 'playing', 'brother', 'learning', 'late', 'child', 'books', 'decided', 'thei', 'imagine',
 'art', 'small', 'huge', 'wait', 'sleep', 'single', 'kids', 'description', 'top' 'song', 'heart', 'infjs', 'hell', 'sad', 'super',
 'human', 'giving', 'wondering', 'value', 'serious', 'youll', 'simply', 'clear', 'break', 'lost', 'learned', 'age', 'typing', 'sister',
 'hand', 'random', 'body', 'run', 'internet', 'living', 'character', 'due', 'hours', 'given', 'difference', 'online', 'telling', 'mentioned',
 'set', 'room', 'constantly', 'games', 'later', 'glad', 'control', 'perhaps','angry', 'ti', 'loved', 'gonna', 'men', 'fear', 'turn',
 'young', 'laugh', 'fall', 'food', 'perc', 'pick', 'confused', 'certainly', 'younger', 'normal', 'outside', 'across', 'physical', 'several',
 'simple', 'infps', 'energy', 'truly', 'intps', 'woman', 'older', 'toi', 'knows', 'aware', 'date', 'lack', 'suppose', 'choose', 'plan',
 'laughing', 'shit', 'kid', 'language', 'theory', 'eat', 'truth', 'gave', 'check', 'cannot', 'tests', 'amazing', 'peoples', 'boyfriend', 'focus',
 'comfortable', 'heres', 'hold','finding', 'respect', 'inside', 'movies', 'hair', 'bored', 'brain', 'situations', 'doubt', 'dominant', 'main',
 'worth', 'state', 'number', 'xd', 'bring', 'reply', 'characters', 'meaning', 'knowing', 'likes', 'nature', 'enfps', 'extroverted',
 'opposite', 'results', 'specific', 'worry', 'afraid', 'black', 'somewhere', 'behind', 'phone', 'process', 'romantic', 'shy', 'quiet','listening',
'power', 'accurate', 'negative', 'cold', 'entps','song','nf','nt','term','introvert','sj','dream', 'ex',
 'realized', 'finally', 'weeks', 'early','particular', 'annoying', 'boring', 'describe', 'vs', 'system', 'points', 'rest', 'english', 'avoid']

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cv2 = CountVectorizer(max_features=1500, min_df=1, max_df=1.0, stop_words=stop_words)
data_cv2 = cv2.fit_transform(data_nouns.posts)
data_dtmn = pd.DataFrame(data_cv2.toarray(), columns=cv2.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cv2.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

# Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [ ]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    word_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if word_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_cleaning.posts.apply(nouns_adj))
data_nouns_adj

In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(max_features=1500, min_df=1, max_df=1.0, stop_words=stop_words)
data_cvna = cvna.fit_transform(data_nouns_adj.posts)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: voice, song
* Topic 1: science, religion
* Topic 2: intuition, logic
* Topic 3: spouse, anxiety 

In [ ]:
# Let's take a look at which topics each post contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))